## Test data

In [9]:
test_ra = 168.3700 #degrees
test_dec = 22.15167 #degrees
test_d = 233000 #pc
mu_a = -6.930000 #mas/cent
mu_d = -8.670000 #mas/cent
rv = 79.1 #km/s

## Python Functions

In [10]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord, Galactocentric, ICRS, Galactic
from astropy.coordinates import BaseCoordinateFrame, FrameAttribute, TimeFrameAttribute, RepresentationMapping
import gala.coordinates as gc
    
#Constants
asec_to_rad = 206264.8 #Converts arcseconds to radians
year_to_sec = 3.155693e7 #Converts from years to seconds
pc_to_km = 3.08568e13 #Converts from pc to km
year_to_day = 365.2422 #Converts years to days
ra_lsr = 318.00425 #RA of the apex of the LSR motion (ie., l=90, b=0)     
dec_lsr = 48.329611 #DEC of the apex of the LSR motion (J2000.0)
ra_sa = 251.51 #RA of the apex of the solar peculiar motion (wrt LSR) J2000.0 
dec_sa = 10.129 #DEC of the apex of the solar peculiar motion (wrt LSR)
alpha_NGP = 192.85948 #RA in degrees of the Galactic Pole (J2000.0)
delta_NGP = 27.12825 #DEC in degrees of the Galactic Pole (J2000.0)
gc_GAN = 32.93192 #Angle along galactic equator from (l,b)=(0,0) to galactic ascending node
l_NCP = 122.9319#Galactic longitude of the NCP (J2000.0)
b_NCP =  27.1283#Galactic latitude of the NCP  (J2000.0)
usun = -10.0 #U-component of Sun's velocity w.r.t. LSR; Dehnen & Binney                  
vsun = 11.0 #V-component of Sun's velocity w.r.t. LSR; 1998, MNRAS, 
wsun = 7.0 #W-component of Sun's velocity w.r.t. LSR; 298, 387
vlsr = 237.0 #Circular velocity of LSR w.r.t. Galaxy center in km/s
dlsr = 8200.0 #Distance of LSR from the Galaxy center in pc

#Functions

def sin_deg(x):
    #Sine function for degree-valued args
    deg_to_rad = 1.74532925199e-02
    return np.sin(np.deg2rad(x))

def cos_deg(x):
    #Cosine function for degree-valued args
    return np.cos(np.deg2rad(x))

def asin_deg(x):
    #Gives asin in degrees
    return np.rad2deg(np.arcsin(x))

def acos_deg(x):
    #Gives acos in degrees
    return np.rad2deg(np.arccos(x))

def atan2_deg(y,x):
    #Gives atan2 in degrees
    return np.rad2deg(np.arctan2(y,x))

def radec_to_lb(ra, dec):
    #Returns dictionary of galactic coords (l, b) in degrees, given an object's RA/DEC in degrees
    coords = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
    galactic = coords.galactic
    return galactic.l.deg, galactic.b.deg

def lb_to_galactic_xyz(l, b, d, dlsr):
    #Takes l and b in degrees, d (distance to object) in pc, dlsr in pc
    x = (dlsr-d*cos_deg(b)*cos_deg(l))/1000
    y = -d*cos_deg(b)*sin_deg(l)/1000
    z = d*sin_deg(b)/1000
    return x, y, z #kpc

def pm_radec_to_lb(ra, dec, dist, mu_a, mu_d, rv):
    #Takes ra/dec in degrees, distance in pc, mu_a/mu_d in mas/cent, rv in km/s
    #Returns mu_l/mu_b in mas/cent
    c = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, distance=dist*u.pc)
    pm = [mu_a, mu_d] * u.mas/(100*u.yr) 
    return gc.pm_icrs_to_gal(c, pm).to(u.mas/(u.yr))

def GETVT(mux,muy,d,asec_to_rad=asec_to_rad,year_to_sec=year_to_sec,pc_to_km=pc_to_km):
    #mux, muy in mas/cent
    #distance in pc
    
    #Get into arcsec/year
    mux = (mux*(1e-3)/100)
    muy = (muy*(1e-3)/100)

    vtx=mux*d*pc_to_km/(year_to_sec*asec_to_rad)
    vty=muy*d*pc_to_km/(year_to_sec*asec_to_rad)
    vt=np.sqrt(vtx*vtx+vty*vty)
    return vtx, vty, vt  

def get_v_wrt_lsr(l, b, vtl, vtb, v_r_sun, usun, vsun, wsun):
    #Finds velocity of galaxy wrt LSR
    #velocity inputs are in km.s
    
    #Theta direction
    vg = v_r_sun*cos_deg(b)*sin_deg(l) - vtb*sin_deg(b)*sin_deg(l)+vtl*cos_deg(l) + vsun
    
    #Pi direction
    ug = -v_r_sun*cos_deg(b)*cos_deg(l) + vtb*sin_deg(b)*cos_deg(l) + vtl*sin_deg(l) + usun
    
    #Z direction
    wg =  v_r_sun*sin_deg(b) + vtb*cos_deg(b) + wsun
    
    return ug, vg, wg #km/s

def get_v_cyl(l,b,d,ug,vg,wg, dlsr=dlsr, vlsr=vlsr):
    #input velocities in km/s, distainces in pc
    
    x=np.sqrt(dlsr*dlsr+d*d*cos_deg(b)*cos_deg(b)-2.0*dlsr*d*cos_deg(b)*cos_deg(l))
    sinalpha=d*cos_deg(b)*sin_deg(l)/x
    cosalpha=(x*x+dlsr*dlsr-d*d*cos_deg(b)*cos_deg(b))/(2.0*x*dlsr)

    vpi =  ug*cosalpha + (vg+vlsr)*sinalpha
    vtheta = -ug*sinalpha + (vg+vlsr)*cosalpha
    vz =  wg
    
    #upg(2) =  sqrt((ug(2)*cosalpha)**2 + (vg(2)*sinalpha)**2)
    #vpg(2) =  sqrt((ug(2)*sinalpha)**2 + (vg(2)*cosalpha)**2)
    #wpg(2) =  wg(2)
    
    return vpi, vtheta, vz #km/s

def get_v_xyz(l, b, d, vpi, vth, vz, dlsr=dlsr):
    #distances in pc, vels in km/s

    x=dlsr-d*cos_deg(b)*cos_deg(l)
    y=-d*cos_deg(b)*sin_deg(l)
    z=d*sin_deg(b)

    costh=x/np.sqrt(x**2+y**2)
    sinth=-y/np.sqrt(x**2+y**2)
    vx=vpi*costh-vth*sinth
    vy=-vpi*sinth-vth*costh
    
    return vx, vy, vz

def get_v_spherical(l, b, d, vpi, vtheta, vz, dlsr=dlsr):
#                 --- The transformation equations: ---
#       uppg =  upg*cos(beta) + wpg*sin(beta)
#       vppg =  vpg
#       wppg = -upg*sin(beta) + wpg*cos(beta)
#
#     --- beta is an angle between the line-of-sight to a galaxy and
#         line x (see subroutine GETUPGVPGWPG for definition).
#     --- From the law of cosines...

    x=np.sqrt(dlsr*dlsr+d*d*cos_deg(b)*cos_deg(b)-2.0*dlsr*d*cos_deg(b)*cos_deg(l))
    cosbeta=x/np.sqrt(x*x+d*d*sin_deg(b)*sin_deg(b))
    sinbeta=d*sin_deg(b)/np.sqrt(x*x+d*d*sin_deg(b)*sin_deg(b))
    vr =  vpi*cosbeta + vz*sinbeta
    vrot = vtheta
    v_perp_radial = -vpi*sinbeta + vz*cosbeta

    vt = np.sqrt(vrot**2+v_perp_radial**2)
    
    return vr, vrot, v_perp_radial, vt


## Print Values

In [18]:
l, b = radec_to_lb(test_ra, test_dec)
print "l = {0}, b = {1}" .format(l,b) +" (deg)" + "\n"

xyz = [i for i in lb_to_galactic_xyz(l, b, test_d, dlsr)]
print "Galactocentric X, Y, Z = {0}" .format(xyz) + " (kpc)" + "\n"

mu_l, mu_b = pm_radec_to_lb(test_ra, test_dec, test_d, mu_a, mu_d, rv)*u.yr/u.mas*100
print "mu_l = {0}, mu_b = {1}" .format (mu_l, mu_b) + " (mas/cent)" + "\n"

vtl, vtb, vt = GETVT(mu_l, mu_b, test_d)
print "v_tang in l direction = {0}, v_tang in b direction = {1}, v_tang total = {2}".format(vtl, vtb, vt) \
+ " (km/s)" + "\n"

ug, vg, wg = get_v_wrt_lsr(l, b, vtl, vtb, rv, usun, vsun, wsun)
print "Velocities of galaxy wrt LSR:"
print "u = {0}, v = {1}, w = {2}" .format(ug, vg, wg) + " (km/s)" + "\n"

vpi, vtheta, vz = get_v_cyl(l,b,test_d,ug,vg,wg)
print "Cylindrical velocities:"
print "vpi = {0}, vtheta = {1}, vz = {2}" .format(vpi, vtheta, vz) + " (km/s)" + "\n"

vx, vy, vz = get_v_xyz(l, b, 233000, vpi, vtheta, vz)
print "Galactocentric XYZ velocities:"
print "Vx = {0}, Vy = {1}, Vz = {2}" .format (vx, vy, vz) + " (km/s)" + "\n"

vr, vrot, v_perp_radial, vt = get_v_spherical(l, b, test_d, vpi, vtheta, vz)
print "Radial velocity = {0}" .format(vr) + " (km/s)"
print "Velocity in direction of disk rotation = {0}" .format(vrot) + " (km/s)"
print "V up and perp to radial = {0}" .format(v_perp_radial) + " (km/s)"
print "Tangential velocity = {0}" .format(vt) + " (km/s)"

l = 220.16913737, b = 67.2312443585 (deg)

Galactocentric X, Y, Z = [77.105875011103947, 58.166385352063095, 214.84331966349203] (kpc)

mu_l = 6.17081529568, mu_b = -9.22577180982 (mas/cent)

v_tang in l direction = 68.1599852049, v_tang in b direction = -101.903628602, v_tang total = 122.597443307 (km/s)

Velocities of galaxy wrt LSR:
u = 41.2271830735, v = -121.440923417, w = 40.498068962 (km/s)

Cylindrical velocities:
vpi = -36.6806351847, vtheta = 117.081645903, vz = 40.498068962 (km/s)

Galactocentric XYZ velocities:
Vx = 41.2271830735, Vy = -115.559076583, Vz = 40.498068962 (km/s)

Radial velocity = 21.896955667 (km/s)
Velocity in direction of disk rotation = 117.081645903 (km/s)
V up and perp to radial = 50.0608222039 (km/s)
Tangential velocity = 127.334982338 (km/s)


## Get UPGVPGWPG

In [12]:
def GETUPGVPGWPG(dlsr,vlsr,d,l,b,ug,vg,wg):
#----------------------------------------------------------------------------
#                 --- The transformation equations: ---
#
#       upg =  ug*cos(alpha) + (vg+vlsr)*sin(alpha) ;Pi direction
#       vpg = -ug*sin(alpha) + (vg+vlsr)*cos(alpha) ;Theta direction
#       wpg =  wg                                   ;Z direction
#
#     ---alpha is an angle between dlsr (a line between the Galactic center
#        and the LSR (dlsr=8500 pc) and x (a line between the Galactic center
#        and a point on the Galactic disk corresponding to a vertical 
#        projection of a galaxy onto the disk. 
#        
#     --- The circular velocity of the L.S.R. is removed.
#----------------------------------------------------------------------------


    x=np.sqrt(dlsr*dlsr+d*d*cos_deg(b)*cos_deg(b)-2.0*dlsr*d*cos_deg(b)*cos_deg(l))
    sinalpha=d*cos_deg(b)*sin_deg(l)/x
    cosalpha=(x*x+dlsr*dlsr-d*d*cos_deg(b)*cos_deg(b))/(2.0*x*dlsr)

    upg =  ug*cosalpha + (vg+vlsr)*sinalpha
    vpg = -ug*sinalpha + (vg+vlsr)*cosalpha
    wpg =  wg
    
    #upg(2) =  sqrt((ug(2)*cosalpha)**2 + (vg(2)*sinalpha)**2)
    #vpg(2) =  sqrt((ug(2)*sinalpha)**2 + (vg(2)*cosalpha)**2)
    #wpg(2) =  wg(2)
    
    return upg, vpg, wpg

upg, vpg, wpg = GETUPGVPGWPG(dlsr,vlsr,test_d,l,b,ug,vg,wg)
print upg, vpg, wpg #km/s

-36.6806351847 117.081645903 40.498068962


## Get Galactocentric tangential velocity components

In [13]:
def GETUPPGVPPGWPPG(dlsr,d,l,b,upg,vpg,wpg):
#                 --- The transformation equations: ---
#       uppg =  upg*cos(beta) + wpg*sin(beta)
#       vppg =  vpg
#       wppg = -upg*sin(beta) + wpg*cos(beta)
#
#     --- beta is an angle between the line-of-sight to a galaxy and
#         line x (see subroutine GETUPGVPGWPG for definition).
#     --- From the law of cosines...

    x=np.sqrt(dlsr*dlsr+d*d*cos_deg(b)*cos_deg(b)-2.0*dlsr*d*cos_deg(b)*cos_deg(l))
    cosbeta=x/np.sqrt(x*x+d*d*sin_deg(b)*sin_deg(b))
    sinbeta=d*sin_deg(b)/np.sqrt(x*x+d*d*sin_deg(b)*sin_deg(b))
    uppg =  upg*cosbeta + wpg*sinbeta
    vppg = vpg
    wppg = -upg*sinbeta + wpg*cosbeta

    vt = np.sqrt(vppg**2+wppg**2)
    
    return uppg, vppg, wppg, vt

GETUPPGVPPGWPPG(dlsr, 233000, l, b, upg, vpg, wpg)

(<Quantity 21.896955667014314>,
 <Quantity 117.0816459030816>,
 <Quantity 50.0608222038629>,
 <Quantity 127.33498233832428>)

In [ ]:
vcyl = u.Quantity([upg, -vpg, wpg], u.km/u.s) #Theta is in negative direction from usual cylindrical theta velocity

from galpy.util.bovy_coords import rect_to_cyl, cyl_to_rect_vec

pi, theta, z = rect_to_cyl(positions[0], positions[1], positions[2])
print pi, theta, z

pcyl = u.Quantity([pi, theta, z], u.kpc)

gc.cylindrical_to_cartesian(pcyl, vcyl)

In [ ]:
cyl_to_rect_vec(vr=upg, vt=-vpg, vz=wpg, phi=

### Using astropy to get XYZ position

In [ ]:
def getxyz(ra, dec, d):
    c = ICRS(ra=ra*u.degree, dec=dec*u.degree, distance=d*u.pc)
    galac = c.transform_to(Galactocentric(galcen_distance=8.1999*u.kpc, z_sun=0*u.pc))
    return galac 
positions = getxyz(test_ra, test_dec, test_d)

poslist = u.Quantity([-positions.x, -positions.y, positions.z], u.kpc)
print poslist

___

## Development

In [16]:
def xyzvelocity(ra, dec, dist, mu_a, mu_d, rv):
    pm = [mu_a, mu_d] * u.uas/(100*u.yr) 
    rv = rv * u.km/u.s
    frame = Galactocentric(z_sun=0*u.pc, galcen_distance=8.2*u.kpc)
    c = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, distance=dist*u.kpc)
    d = gc.vhel_to_gal(c.icrs, pm=pm, rv=rv, galactocentric_frame=frame, vlsr=[-usun, -vsun, wsun]*u.km/u.s)
    return d

xyzvels = xyzvelocity(test_ra, test_dec, test_d, mu_a, mu_d, rv)


print xyzvels, type(xyzvels)


[-41.22674788  76.56157507  40.49867244] km / s <class 'astropy.units.quantity.Quantity'>


## Mathematical Versions

In [17]:
def radec_to_galactic_pos2(ra, dec, alpha_NGP=alpha_NGP, delta_NGP=delta_NGP, gc_GAN=gc_GAN):
    #Finds galactic coords (l, b) of an object given RA, DEC in degrees.       
    b = asin_deg(sin_deg(dec)*cos_deg(90.0-delta_NGP)-cos_deg(dec)*sin_deg(ra-(alpha_NGP+90.0))*sin_deg(90.0-delta_NGP))
    y = cos_deg(dec)*sin_deg(ra-(alpha_NGP+90.0))*cos_deg(90.0-delta_NGP)+sin_deg(dec)*sin_deg(90.0-delta_NGP)
    x = cos_deg(dec)*cos_deg(ra-(alpha_NGP+90.0))
    l = atan2_deg(y,x)+gc_GAN
    if l < 0:
        l += 360
    return {'l':l, 'b':b}

radec_to_galactic_pos2(test_ra, test_dec)

{'b': 67.231242804005689, 'l': 220.16912488386765}

## Deprecated

## astropy xyz velocity

In [ ]:
def xyzvelocity(ra, dec, dist, mu_a, mu_d, rv):
    pm = [mu_a, mu_d] * u.uas/(100*u.yr) 
    rv = rv * u.km/u.s
    frame = Galactocentric(z_sun=0*u.pc, galcen_distance=8.2*u.kpc)
    c = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, distance=dist*u.kpc)
    d = gc.vhel_to_gal(c.icrs, pm=pm, rv=rv, galactocentric_frame=frame, vlsr=[usun, vsun, wsun]*u.km/u.s)
    return d

xyzvels = xyzvelocity(test_ra, test_dec, test_d, mu_a, mu_d, rv)

print xyzvels, type(xyzvels)

## New frame

In [ ]:
from astropy.coordinates import frame_transform_graph
from astropy.coordinates.angles import rotation_matrix
import astropy.coordinates as coord
import astropy.units as u

class RealGalactocentric(coord.BaseCoordinateFrame):
  
    default_representation = coord.CartesianRepresentation

@frame_transform_graph.transform(coord.FunctionTransform, coord.Galactocentric, RealGalactocentric)
def galactocentric_to_realgalactocentric(galactocentric, realgalactocentric):

    x = np.atleast_1d(galactocentric.x)
    y = np.atleast_1d(galactocentric.y)
    z = np.atleast_1d(galactocentric.z)
    X = -x
    Y = -y
    Z = z
        
    return RealGalactocentric(x=X, y=Y, z=Z)
    
@frame_transform_graph.transform(coord.FunctionTransform, RealGalactocentric, coord.Galactocentric)
def realgalactocentric_to_galactocentric(realgalactocentric, galactocentric):

    X = np.atleast_1d(realgalactocentric.x)
    Y = np.atleast_1d(realgalactocentric.y)
    Z = np.atleast_1d(realgalactocentric.z)
    
    x = -X
    y = -Y
    z = z

    return coord.Galactocentric(x=x, y=y, z=z)

galacto = coord.Galactocentric(x=-77*u.kpc, y=-58*u.kpc, z=214*u.kpc)
galacto.transform_to(RealGalactocentric)

In [ ]:
from astropy.coordinates.builtin_frames.galactocentric import _ROLL0 as ROLL0
try:
    from astropy.coordinates.matrix_utilities import rotation_matrix, matrix_transpose, matrix_product
    ASTROPY_1_3 = True
except ImportError:
    from .matrix_utilities import rotation_matrix, matrix_transpose, matrix_product
    ASTROPY_1_3 = False

if not ASTROPY_1_3:
    import astropy
    import warnings
    warnings.warn("We recommend using Astropy v1.3 or later. You have: {}"
                  .format(astropy.__version__), DeprecationWarning)

def _icrs_gctc_velocity_matrix(galactocentric_frame):
    """
    Construct a transformation matrix to go from heliocentric ICRS to a galactocentric
    frame. This is just a rotation and tilt which makes it approximately the same
    as transforming to Galactic coordinates. This only works for velocity because there
    is no shift due to the position of the Sun.
    """

    # define rotation matrix to align x(ICRS) with the vector to the Galactic center
    M1 = rotation_matrix(-galactocentric_frame.galcen_dec, 'y')
    M2 = rotation_matrix(galactocentric_frame.galcen_ra, 'z')

    # extra roll away from the Galactic x-z plane
    M3 = rotation_matrix(ROLL0 - galactocentric_frame.roll, 'x')

    # rotate about y' to account for tilt due to Sun's height above the plane
    z_d = (galactocentric_frame.z_sun / galactocentric_frame.galcen_distance).decompose()
    M4 = rotation_matrix(-np.arcsin(z_d), 'y')

    return matrix_product(M4, M3, M1, M2)

VCIRC = 220.*u.km/u.s
VLSR = [10., 5.25, 7.17]

def vhel_to_gal(coordinate, pm, rv, vcirc=VCIRC, vlsr=VLSR, galactocentric_frame=None):

    if galactocentric_frame is None:
        galactocentric_frame = coord.Galactocentric

    # make sure this is a coordinate and get the frame for later use
    c = coordinate

    if c.name == 'icrs':
        pm_radec = u.Quantity(map(np.atleast_1d,pm))
        icrs = c

    else:
        pm_radec = transform_proper_motion(c, pm, coord.ICRS)
        icrs = c.transform_to(coord.ICRS)

    # I'm so fired
    a,d,D = icrs.ra, icrs.dec, c.distance

    # proper motion components: longitude, latitude
    mura_cosdec, mudec = pm_radec
    vra = (D*mura_cosdec).to(rv.unit, equivalencies=u.dimensionless_angles())
    vdec = (D*mudec).to(rv.unit, equivalencies=u.dimensionless_angles())

    v_icrs = [rv*np.cos(a)*np.cos(d) - vra*np.sin(a) - vdec*np.cos(a)*np.sin(d),
              rv*np.sin(a)*np.cos(d) + vra*np.cos(a) - vdec*np.sin(a)*np.sin(d),
              rv*np.sin(d) + vdec*np.cos(d)]
    v_icrs = np.array([v.to(u.km/u.s).value for v in v_icrs]) * u.km/u.s
    
    R = _icrs_gctc_velocity_matrix(galactocentric_frame)

    orig_shape = v_icrs.shape
    v_gc = R.dot(v_icrs.reshape(v_icrs.shape[0], -1)).reshape(orig_shape)

    #Correct for reverse sign on x and y axes
    v_gc[0]=-v_gc[0]
    v_gc[1]=-v_gc[1]
    
    # remove circular and LSR velocities
    v_gc[1] = v_gc[1] + vcirc #Changed from plus sign
    for i in range(3):
        v_gc[i] = v_gc[i] + vlsr[i] #originally plus sign

    if c.isscalar:
        return v_gc.reshape((3,))
    else:
        return v_gc

In [ ]:
def xyzvelocity(ra, dec, dist, mu_a, mu_d, rv):
    pm = [mu_a, mu_d] * u.uas/(100*u.yr) 
    rv = rv * u.km/u.s
    frame = Galactocentric(z_sun=0*u.pc, galcen_distance=8.2*u.kpc)
    c = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, distance=dist*u.kpc)
    d = vhel_to_gal(c.icrs, pm=pm, rv=rv, galactocentric_frame=frame, vlsr=[usun, vsun, wsun]*u.km/u.s)
    return d

xyzvels = xyzvelocity(test_ra, test_dec, test_d, mu_a, mu_d, rv)


print xyzvels, type(xyzvels)

In [ ]:
from galpy.util.bovy_coords import vrpmllpmbb_to_vxvyvz, vxvyvz_to_galcenrect, rect_to_cyl

vxvyvz = vrpmllpmbb_to_vxvyvz(79.1, 0.061708153, -0.092257718, 220.16913737, 67.2312443585, 233, XYZ=False, degree=True)
print vxvyvz


vxvyvz_to_galcenrect(vxvyvz[0], vxvyvz[1], vxvyvz[2], [usun, vsun, wsun], 8.2, 0)
